In [ ]:
import matplotlib.image as img
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import numpy as np 
from PIL import Image
import cv2


In [ ]:
def show_image(image, title='Image', cmap_type = "gray") :
  plt.imshow(image, cmap = cmap_type)
  plt.axis('off')
  plt.title(title)
  plt.show()

In [ ]:
# grayscaled_image = np.array(Image.open("a/cameraman.jpg").convert('L'))

"""useful prints"""
#show_image(im)
#print(type(im))

#with np.printoptions(threshold=np.inf):
#  print(grayscaled_image) 

'useful prints'

In [ ]:
def convolution(image, kernel):
    # Flip the kernel
    kernel = np.flip(kernel,0)
    kernel = np.flip(kernel,1)

    # Determine output image matrix size. Subtract Kernel Size (3) and add 1 to the original image
    output_image_column_number = image.shape[0] - 2
    output_image_row_number    = image.shape[1] - 2

    #create output image matrix
    output_image = np.zeros((output_image_column_number, output_image_row_number))

    for y in range( output_image_row_number):
        for x in range( output_image_column_number):     
          list = kernel * image[x: x + 3, y: y + 3]
          output_image[x, y] = list.sum() 
          

    return output_image

"""
  # Create output matrix
  suppressed_image = np.zeros(image.shape)
  rows, cols = suppressed_image.shape[0], suppressed_image.shape[1]

  for i in range(rows - 1):
    for j in range(cols - 1):
        
      if (gradient_direction[i][j] >= 337.5 or gradient_direction[i][j] < 22.5) or (gradient_direction[i][j] >= 157.5 and gradient_direction[i][j] < 202.5):
        if image[i][j] >= image[i][j + 1] and image[i][j] >= image[i][j - 1]:
          suppressed_image[i][j] = image[i][j]
  
      if (gradient_direction[i][j] >= 22.5 and gradient_direction[i][j] < 67.5) or (gradient_direction[i][j] >= 202.5 and gradient_direction[i][j] < 247.5):
        if image[i][j] >= image[i - 1][j + 1] and image[i][j] >= image[i + 1][j - 1]:
          suppressed_image[i][j] = image[i][j]

      if (gradient_direction[i][j] >= 67.5 and gradient_direction[i][j] < 112.5) or (gradient_direction[i][j] >= 247.5 and gradient_direction[i][j] < 292.5):
        if image[i][j] >= image[i - 1][j] and image[i][j] >= image[i + 1][j]:
          suppressed_image[i][j] = image[i][j]
      # 135 degrees
      if (gradient_direction[i][j] >= 112.5 and gradient_direction[i][j] < 157.5) or (gradient_direction[i][j] >= 292.5 and gradient_direction[i][j] < 337.5):
        if image[i][j] >= image[i - 1][j - 1] and image[i][j] >= image[i + 1][j + 1]:
          suppressed_image[i][j] = image[i][j]

  return suppressed_image
  """

In [ ]:
def non_maximum_suppression(image, direction):

    rows, cols = image.shape[0], image.shape[1]
    result = image.copy()

    for x in range(rows):
      for y in range(cols):
          # 0 degrees
          if (direction[x][y] < 22.5 and direction[x][y] >= 0) or (direction[x][y] >= 157.5 and direction[x][y] < 202.5) or 
          (direction[x][y] >= 337.5 and direction[x][y] <= 360):
              direction[x][y] = 0
          # 45 degrees
          elif (direction[x][y] >= 22.5 and direction[x][y] < 67.5) or (direction[x][y] >= 202.5 and direction[x][y] < 247.5):
              direction[x][y] = 45
          # 90 degrees
          elif (direction[x][y] >= 67.5 and direction[x][y] < 112.5) or (direction[x][y] >= 247.5 and direction[x][y] < 292.5):
              direction[x][y] = 90
          # 135 degrees
          else:
              direction[x][y] = 135

    for x in range(1, rows-1):
      for y in range(1, cols-1):
          # 0 degrees
          if direction[x][y] == 0:
              if (result[x][y] <= result[x][y+1]) or (result[x][y] <= result[x][y-1]):
                  result[x][y] = 0
          # 45 degrees
          elif direction[x][y] == 45:
              if (result[x][y] <= result[x-1][y+1]) or (result[x][y] <= result[x+1][y-1]):
                  result[x][y] = 0
          # 90 degrees
          elif direction[x][y] == 90:
              if (result[x][y] <= result[x+1][y]) or (result[x][y] <= result[x-1][y]):
                  result[x][y] = 0
          # 135 degrees
          elif direction[x][y] == 135:
              if (result[x][y] <= result[x+1][y+1]) or (result[x][y] <= result[x-1][y-1]):
                  result[x][y] = 0

    return result

SyntaxError: ignored

In [ ]:
def double_thresholding(image):

    # Create resulting image
    output_image  = np.zeros(image.shape)
    strong = 255
    weak   = 75
    mmax = np.max(image)
    print(mmax)
    low_threshold = 0.1 * mmax
    high_threshold = 0.4 * mmax
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i][j] >= high_threshold:
                output_image[i][j] = strong
            elif image[i][j] >= low_threshold:
                output_image[i][j] = weak
    
    #with np.printoptions(threshold=np.inf):
    #  print(output_image) 

    return output_image

In [ ]:
if __name__ == '__main__':
    # Convert to Grayscale Image
    image = np.array(Image.open("a/cameraman.jpg").convert('L'))
    # Gaussian Kernel
    # kernel = np.array([[1, 4, 7, 4, 1],  [4, 16, 26, 16, 4], [7, 26, 41, 26, 7], [4, 16, 26, 16, 4], [1, 4, 7, 4, 1]])
    kernel = np.array([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])

    # Smooth image
    smoothed_image = convolution(image, kernel)

    # Edge Detection Kernels
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    # Take gradients for both x and y axes
    gradient_x = convolution(smoothed_image, Kx)
    gradient_y = convolution(smoothed_image, Ky)

    # Calculate gradient magnitude matrix
    gradient_magnitude = np.sqrt(gradient_x**2+ gradient_y**2)

    # Calculate gradient direction matrix
    gradient_direction = np.arctan2(gradient_y, gradient_x) * 180 / np.pi

    # Replace the negative pi angle values with their absolute value
    gradient_direction[gradient_direction < 0] += 180

    # print(Theta)
    
    # show_image(output)
    # cv2.imwrite('2DConvolved.bmp', output)

In [ ]:
suppressed_image = non_maximum_suppression(gradient_magnitude, gradient_direction)

In [ ]:
thresholded_image = double_thresholding(suppressed_image)

In [ ]:
def hysteresis(image, strong = 255, weak = 75):
    
    M, N = image.shape  
    
    for i in range(1, M-1):
        for j in range(1, N-1):
            if image[i,j] == weak:              
                if image[i+1, j-1] == strong or image[i+1, j] == strong or image[i+1, j+1] == strong\
                        or image[i, j-1] == strong or image[i, j+1] == strong \
                        or image[i-1, j-1] == strong or image[i-1, j] == strong or image[i-1, j+1] == strong:
                  image[i, j] = 255
                else:
                  image[i, j] = 0

    return image

In [ ]:
canny_edge_image = hysteresis(thresholded_image)

In [ ]:
 show_image(canny_edge_image)
 #print(type(canny_edge_image))

In [ ]:
import imageio
imageio.imwrite('canny.png', canny_edge_image)
imageio.imwrite('thres.png', thresholded_image)
imageio.imwrite('gradient_x.png', gradient_x)
imageio.imwrite('gradient_y.png', gradient_y)

